##Object Length Dataframe

Development Envrionment
<br/>xml Parsing
<br/>Object Length Calculation
<br/>Detailed Class Polygon Dataframe
<br/>Class Polygon Dataframe

#Development Envrionment

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import cv2
import random
import pandas as pd
from glob import glob
from pandas import option_context
from termcolor import colored
import matplotlib.pyplot as plt
from matplotlib import gridspec
from IPython.display import HTML
from IPython.display import display

In [ ]:
import xml.etree.ElementTree as ET
import xml.etree.ElementTree

#xml Parsing

In [ ]:
xml_folder_path = '/content/drive/MyDrive/Aiffelthon/eda/xml_data'
xml_folder_list = [xml_folder_path + '/train/train_part1_xml', xml_folder_path + '/train/train_part2_xml',
                   xml_folder_path + '/validation/vaildation_xml']

In [ ]:
def xml_file_list(xml_folder):
  label = glob(xml_folder+'/labelXml'+'/*')
  label = sorted(label, reverse=False)
  label = sorted(label, key=len)
  return label

In [ ]:
for i in xml_folder_list:
  if 'train_part1' in i:
    train_part1_label = xml_file_list(i)
  elif 'train_part2' in i:
    train_part2_label = xml_file_list(i)
  elif 'validation' in i:
    validation_label = xml_file_list(i)

In [ ]:
train_label = train_part1_label + train_part2_label

In [ ]:
tree = ET.parse(train_part1_label[0])
root = tree.getroot()
ET.dump(tree)

<annotation>
	<source>
		<filename>0.tif</filename>
		<origin>GF2/GF3</origin>
	</source>
	<research>
		<version>1.0</version>
		<provider>FAIR1M</provider>
		<author>Cyber</author>
		<pluginname>FAIR1M</pluginname>
		<pluginclass>object detection</pluginclass>
		<time>2021-07-21</time>
	</research>
	<size>
		<width>1500</width>
		<height>1500</height>
		<depth>3</depth>
	</size>
	<objects>
		<object>
			<coordinate>pixel</coordinate>
			<type>rectangle</type>
			<description>None</description>
			<possibleresult>
				<name>Liquid Cargo Ship</name>
			</possibleresult>
			<points>
				<point>1275.000000,458.000000</point>
				<point>1494.000000,88.000000</point>
				<point>1417.000000,43.000000</point>
				<point>1199.000000,414.000000</point>
				<point>1275.000000,458.000000</point>
			</points>
		</object>
		<object>
			<coordinate>pixel</coordinate>
			<type>rectangle</type>
			<description>None</description>
			<possibleresult>
				<name>Passenger Ship</name>
			</possibleresult>


#Object Length Calculation

In [ ]:
tree = ET.parse(train_label[0])
root = tree.getroot()
print(root[0][0].text)
print()

class_name_list = []
file_point_list = []

for i in root[3]:
  for child in i[3]:
    class_name_list.append(child.text)
    class_point_list = []
    for j in range(len(i[4])):
      class_point_tuple = []
      points = i[4][j].text.split(',')
      for point in points:
        point = float(point)
        class_point_tuple.append(point)
      class_point_tuple = tuple(class_point_tuple)
      class_point_list.append(class_point_tuple)
    file_point_list.append(class_point_list)     

for i, j in zip(class_name_list, file_point_list):
  print(i, j)

0.tif

Liquid Cargo Ship [(1275.0, 458.0), (1494.0, 88.0), (1417.0, 43.0), (1199.0, 414.0), (1275.0, 458.0)]
Passenger Ship [(630.0, 403.0), (586.0, 385.0), (582.0, 397.0), (625.0, 414.0), (630.0, 403.0)]
Dry Cargo Ship [(591.0, 1531.0), (761.0, 1377.0), (718.0, 1332.0), (549.0, 1486.0), (591.0, 1531.0)]


[TUTORIAL 2: CUSTOMIZE DATASETS](https://mmrotate.readthedocs.io/en/latest/tutorials/customize_dataset.html)

The annotation txt files in DOTA format:
<br/>Polygons with format (x1, y1, x2, y2, x3, y3, x4, y4)

[How to calculate area of a polygon with latitude and longitude?](https://stackoverflow.com/questions/45733838/how-to-calculate-area-of-a-polygon-with-latitude-and-longitude)

In [ ]:
import tifffile as tiff
from shapely.geometry import Polygon

In [ ]:
object_polygon_list = []
object_length_list = []

for points in file_point_list:
  object_polygon = Polygon(points)
  object_length = object_polygon.length
  object_polygon_list.append(object_polygon)
  object_length_list.append(object_length)

In [ ]:
for i, j, k in zip(class_name_list, file_point_list, object_length_list):
  print(i, j, k)

Liquid Cargo Ship [(1275.0, 458.0), (1494.0, 88.0), (1417.0, 43.0), (1199.0, 414.0), (1275.0, 458.0)] 1037.265872144639
Passenger Ship [(630.0, 403.0), (586.0, 385.0), (582.0, 397.0), (625.0, 414.0), (630.0, 403.0)] 118.51012599707451
Dry Cargo Ship [(591.0, 1531.0), (761.0, 1377.0), (718.0, 1332.0), (549.0, 1486.0), (591.0, 1531.0)] 581.819737695256


#Polygon Length Dataframe

In [ ]:
def class_init():
  vehicle_class_list = ['Dump Truck', 'Cargo Truck', 'Truck Tractor', 'Tractor','Excavator', 'Trailer', 'Van', 'Small Car', 'Bus', 'other-vehicle']
  ship_class_list = ['Dry Cargo Ship', 'Liquid Cargo Ship', 'Motorboat', 'Fishing Boat', 'Tugboat', 'Engineering Ship', 'Warship', 'Passenger Ship', 'other-ship']
  airplane_class_list = ['A220', 'A321', 'A330',  'A350', 'Boeing737', 'Boeing747', 'Boeing777', 'Boeing787', 'ARJ21', 'C919', 'other-airplane']
  
  class_list = [vehicle_class_list, ship_class_list, airplane_class_list]

  polygon_length_df = pd.DataFrame({'file':['a'], 'class':['a'], 'object length':[0.0], 
                                    'point':[[(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]]})

                                  
  return vehicle_class_list, ship_class_list, airplane_class_list, class_list, polygon_length_df

In [ ]:
def namestr(obj, namespace):
    return [name for name in namespace if namespace[name] is obj]

In [ ]:
def polygon_length(label, class_list, polygon_length_df):
  file_name_list = []
  class_name_list = []
  file_point_list = []

  for i in label:
    tree = ET.parse(i)
    root = tree.getroot()
    for class_name in class_list:
      list_index = class_list.index(class_name)
      for detailed_class_name in class_name: 
        for i in root[3]:
          for child in i[3]:
            if child.text == detailed_class_name:
              class_name_list.append(class_name)
              class_point_list = []
              for j in range(0, 4):
                class_point_tuple = []
                points = i[4][j].text.split(',')
                for point in points:
                  point = float(point)
                  class_point_tuple.append(point)
                class_point_tuple = tuple(class_point_tuple)
                class_point_list.append(class_point_tuple)
              file_point_list.append(class_point_list)     
              file_name_list.append(root[0][0].text)

  object_length_list = []
  for points in file_point_list:
    object_polygon = Polygon(points)
    object_length = object_polygon.length
    object_length_list.append(object_length)
  
  count = -1
  for i, j, k, l in zip(file_name_list, class_name_list, object_length_list, file_point_list):
    count += 1
    for m in class_list:
      list_index = class_list.index(m)
      if j == class_list[list_index]:
        if list_index == 0:
          polygon_length_df.loc[count] = [i, 'vehicle', k, l]
        if list_index == 1:
          polygon_length_df.loc[count] = [i, 'ship', k, l]
        if list_index == 2:
          polygon_length_df.loc[count] = [i, 'airplane', k, l]

  if 'train' in namestr(label, globals())[1]:
    df_name = 'train_' + 'polygon_length_df'
  elif 'validation' in namestr(label, globals())[1]:
    df_name = 'validation_' + 'polygon_length_df'
  globals()[df_name] = polygon_length_df  
  globals()[df_name].to_csv('/content/drive/MyDrive/Aiffelthon/eda/object_length/' + df_name + '.csv', index = False)

namestr(label, globals())[ 0 ]이거나 namestr(label, globals())[ 1 ]이다.
<br/>train_label, validation_label에 따라 다르다.

In [ ]:
vehicle_class_list, ship_class_list, airplane_class_list, class_list, polygon_length_df = class_init()
polygon_length(train_label, class_list, polygon_length_df)

In [ ]:
vehicle_class_list, ship_class_list, airplane_class_list, class_list, polygon_length_df = class_init()
polygon_length(validation_label, class_list, polygon_length_df)

In [ ]:
csv_files = glob('/content/drive/MyDrive/Aiffelthon/eda/object_length/' + '*.csv')
  
for f in csv_files:
  df = pd.read_csv(f)
  file_name = ' '.join(s for s in f.split("/")[-1][:-4].split("_")[0:1])
  if file_name == 'an':
    print(f.split("/")[-1].strip(".csv"))
  print(colored(file_name, attrs=['bold']))
  print()
  pd.set_option('display.max_rows', 10)
  with option_context('display.max_colwidth', None):
    display(df)
    print("\n" * 2)

train



,file,class,object length,point
0,0.tif,ship,581.819738,"[(591.0, 1531.0), (761.0, 1377.0), (718.0, 1332.0), (549.0, 1486.0)]"
1,0.tif,ship,1037.265872,"[(1275.0, 458.0), (1494.0, 88.0), (1417.0, 43.0), (1199.0, 414.0)]"
2,0.tif,ship,118.510126,"[(630.0, 403.0), (586.0, 385.0), (582.0, 397.0), (625.0, 414.0)]"
3,1.tif,vehicle,45.254834,"[(43.000004, 953.000061), (38.000004, 958.000061), (27.000004, 947.000061), (32.000004, 942.000061)]"
4,1.tif,vehicle,59.658893,"[(163.124451, 965.919556), (146.000061, 977.0), (140.875671, 969.080444), (158.000061, 958.0)]"
...,...,...,...,...
379412,16485.tif,airplane,170.306642,"[(104.0, 332.0), (78.0, 364.0), (44.0, 335.0), (71.0, 304.0)]"
379413,16486.tif,airplane,319.812837,"[(177.0, 830.0), (238.0, 773.0), (186.0, 717.0), (125.0, 774.0)]"
379414,16487.tif,airplane,179.008862,"[(773.0, 273.0), (745.0, 307.0), (710.0, 277.0), (737.0, 244.0)]"
379415,16487.tif,airplane,194.082617,"[(305.0, 540.0), (277.0, 572.0), (236.0, 536.0), (265.0, 504.0)]"





validation



,file,class,object length,point
0,0.tif,ship,857.939499,"[(841.0, 127.0), (489.0, 20.0), (472.0, 78.0), (824.0, 186.0)]"
1,1.tif,vehicle,76.562244,"[(64.0, 826.0), (55.0, 850.0), (43.0, 846.0), (52.0, 822.0)]"
2,1.tif,vehicle,123.061797,"[(880.0, 775.0), (830.0, 776.0), (830.0, 764.0), (879.0, 763.0)]"
3,1.tif,vehicle,82.067391,"[(866.0, 757.0), (837.0, 762.0), (835.0, 751.0), (865.0, 746.0)]"
4,1.tif,vehicle,156.150410,"[(690.0, 722.0), (623.0, 724.0), (622.0, 713.0), (689.0, 711.0)]"
...,...,...,...,...
193665,8286.tif,ship,283.811249,"[(973.0, 404.0), (851.0, 418.0), (853.0, 437.0), (975.0, 423.0)]"
193666,8286.tif,ship,285.138983,"[(979.0, 422.0), (859.0, 439.0), (862.0, 460.0), (983.0, 442.0)]"
193667,8286.tif,ship,161.276040,"[(722.0, 151.0), (700.0, 200.0), (725.0, 210.0), (747.0, 161.0)]"
193668,8286.tif,ship,166.330784,"[(983.0, 345.0), (914.0, 350.0), (914.0, 363.0), (984.0, 359.0)]"
